In [11]:
import os
from math import pi
from typing import Union
from langchain.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [6]:

# openai_api_key = os.getenv('OPENAI_API_KEY')
# llm = ChatOpenAI(openai_api_key = openai_api_key,temperature = 0.2,model_name = 'gpt-3,5-turbo')
# conversational_memory = ConversationBufferWindowMemory(memory_key='chat_history', k = 5, return_messages= True)



In [12]:
openai_api_key = os.environ.get('OPENAI_API_KEY')
llm = ChatOpenAI(openai_api_key = openai_api_key, temperature = 0, model_name = 'gpt-3.5-turbo')
conversational_memory = ConversationBufferWindowMemory(memory_key= 'chat_history', k= 5, return_messages=True)

In [13]:
class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        return float(radius)*2.0*pi
    
    def _arun(self, radius: Union[int, float]):
        raise NotImplementedError("This tool does not support async")
     


In [14]:
from langchain.agents import initialize_agent
tools = [CircumferenceTool()]
agent = initialize_agent(
    agent= 'chat-conversational-react-description',
    llm = llm,
    tools= tools,
    verbose = True,
    max_iterations = 3, 
    early_stopping_methods = 'generate',
    memory = conversational_memory
    )

In [15]:

agent("can you calculate the circumference of a circle that has a radius of 7.81mm")
# 



> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference calculator",
    "action_input": "7.81"
}
```
Observation: 49.071677249072565
Thought:```json
{
    "action": "Final Answer",
    "action_input": "49.071677249072565"
}
```

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81mm',
 'chat_history': [],
 'output': '49.071677249072565'}

In [ ]:
(7.81 * 2 )* pi

In [ ]:
# If a model tries to not use out tool we can to it like this:
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

In [ ]:
sys_msg = """
Assistant is a large language model trained by OpenAI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

In [ ]:
prompt = agent.agent.create_prompt(system_message=sys_msg, tools=tools)
agent.agent.llm_chain.prompt = prompt

In [ ]:

agent("can you calculate the circumference of a circle that has a radius of 7.81mm")

In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

hf = "Salesforce/blip-image-captioning-large"
processor = BlipProcessor.from_pretrained(hf)
model = BlipForConditionalGeneration.from_pretrained(hf)


In [ ]:
import requests
from PIL import Image
url = 'https://images.unsplash.com/photo-1616128417859-3a984dd35f02?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=2372&q=80' 
image = Image.open(requests.get(url, stream=True).raw).convert('RGB')
image
     

In [ ]:
inputs = processor(image, return_tensors = "pt")
out = model.generate(**inputs, max_new_tokens = 100)
print(processor.decode(out[0]),)

In [ ]:
desc = (
    "use this tool when given the URL of an image that you'd like to be "
    "described. It will return a simple caption describing the image."
)

class ImageCaptionTool(BaseTool):
    name = "Image captioner"
    description = desc

    def _run(self, url: str):
        image = Image.open(requests.get(url, stream=True).raw).convert('RGB')
        inputs = processor(image, return_tensors="pt")
        out = model.generate(**inputs, max_new_tokens=20)
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

tools = [ImageCaptionTool()]

In [ ]:
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt
# update the agent tools
agent.tools = tools








In [ ]:

url_2 = "https://uk.bimago.media/mediacache/catalog/product/cache/6/6/97366/image/750x1120/cb8f05cd4a91169b2d2684134b74881d/97366_1.jpg"
agent(f"What does this image show?{url_2}")